In [1]:
import sys  
sys.path.insert(0, '../py')
from graviti import *

from numpy.linalg import norm
import numpy as np
import os
import os.path
from os import path
import sys
import glob
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('Agg')
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px
import hdbscan
import pandas as pd
import umap
import networkx as nx
from scipy import sparse, linalg
import pickle
from sklearn.preprocessing import normalize, scale
from scipy.sparse import find
from numpy.linalg import norm
import timeit
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

In [2]:
size = 100000 # number of nuclei, use 0 value for full set
nn = 10 # set the number of nearest neighbor in the umap-graph. Will be used in CovD as well

In [3]:
samples = glob.glob('/media/garner1/hdd2/TCGA_polygons/luad/*.gz')    

In [4]:
# Get numb of cores
num_cores = multiprocessing.cpu_count() # numb of cores

In [5]:
for dirpath in samples[:1]:
    sample = os.path.basename(dirpath).split(sep='.')[0]; print(sample)

    print('Loading the data')
    df = pd.DataFrame()
    fovs = glob.glob(dirpath+'/*/*.svs/*.pkl')
    for fov in fovs:
        data = pd.read_pickle(fov)
        df = df.append(data, ignore_index = True)
    df['area'] = df['area'].astype(float)
    
    print('With '+str(df.shape[0])+' nuclei')
    
    features = df.columns[2:];# print(features)
    centroids = df.columns[:2];# print(centroids)

    print('Downsampling '+str(size)+' nuclei')
    if size == 0:
        fdf = df 
    else:
        fdf = df.sample(n=size) 
    pos = fdf[centroids].to_numpy() # Get the positions of centroids 
    fdf = fdf.rename(columns={"Centroid X µm": "cx", "Centroid Y µm": "cy"})
    
    print('Creating the UMAP graph')
    A = space2graph(pos,nn)
    
    print('Finding the neighborhood of the sampled nodes')
    X = df[centroids].to_numpy() # the full array of position
    n_neighbors = df.shape[0]//size + 10
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree',n_jobs=-1).fit(X) 
    distances, indices = nbrs.kneighbors(X) 

    data = scale(df[features].to_numpy(), with_mean=False) #get the morphological data and rescale the data by std 
    
    # Parallel generation of the local covd
    print('Generating the descriptor')
    processed_list = Parallel(n_jobs=num_cores)(
        delayed(covd_parallel_sparse)(node,data,indices) for node in tqdm(list(fdf.index))
                                                   )    
    # Construct the descriptor array
    descriptor = np.zeros((len(processed_list),processed_list[0][1].shape[0]))
    for r in range(len(processed_list)):
        descriptor[r,:] = processed_list[r][1] # covd descriptors of the connected nodes
        
    # Get info about the graph
    row_idx, col_idx, values = find(A) #A.nonzero() # nonzero entries
    print('Generating the diversity index')
    node_nn_diversity = Parallel(n_jobs=num_cores)(
        delayed(covd_gradient_parallel)(node,descriptor,row_idx,col_idx,values) for node in tqdm(range(descriptor.shape[0]))
    )
    fdf['diversity'] = [sum(node_nn_diversity[node][2]) for node in range(descriptor.shape[0])]

    filename = './'+str(sample)+'.size'+str(size)+'.graphNN'+str(nn)+'.covdNN'+str(n_neighbors)+'.nodeHI.pkl'
    fdf.to_pickle(filename)

    print('Generating the edge diversity index')
    edges_list = Parallel(n_jobs=num_cores)(
        delayed(edge_diversity_parallel)(node,neightbors,diversity,fdf) 
                               for (node, neightbors, diversity) in tqdm(node_nn_diversity)
    )
    edge_list = [item for sublist in edges_list for item in sublist]
    edge_df = pd.DataFrame(edge_list, columns=["centroid_x", "centroid_y","diversity"]) 
    filename = './'+str(sample)+'.size'+str(size)+'.graphNN'+str(nn)+'.covdNN'+str(n_neighbors)+'.edgeHI.pkl'
    edge_df.to_pickle(filename)

TCGA-44-7659-01Z-00-DX1
Loading the data
With 434262 nuclei
Downsampling 100000 nuclei
Creating the UMAP graph
Finding the neighborhood of the sampled nodes


  0%|          | 0/100000 [00:00<?, ?it/s]

Generating the descriptor


  0%|          | 240/100000 [00:00<00:42, 2361.10it/s]

Generating the diversity index


  0%|          | 0/100000 [00:00<?, ?it/s]

Generating the edge diversity index


100%|██████████| 100000/100000 [01:39<00:00, 1004.64it/s]


In [6]:
#Show contour plot
N = 100
filename = 'test'
contourPlot(fdf,N,np.mean,filename)

In [13]:
#Show contour plot
N = 20
filename = 'test'
contourPlot(edge_df[edge_df['diversity']<10],N,np.sum,filename)

In [8]:
N = 100
fdf['x_bin'] = pd.cut(fdf['centroid_x'], 2*N, labels=False) # define the x bin label
fdf['y_bin'] = pd.cut(fdf['centroid_y'], N, labels=False) # define the y bin label

table = pd.pivot_table(fdf,
                       values='diversity',
                       index=['x_bin'],
                       columns=['y_bin'],
                       aggfunc=np.sum, # take the mean of the entries in the bin
                       fill_value=None
                      )
import numpy as np
from skimage.io import imsave, imread

image = np.array(np.nan_to_num(table.to_numpy()), dtype=np.uint8)

imsave("test.png", image)
print("image shape")
print(image.shape)

image shape
(23, 11)
